In [ ]:
# 先查看一下下载的一个数据集  safety-belt dataset:archive

# dataset-root D:\ddesktop\monitoring\datadata\safetyBelt\aichive\VOCDatasets
# \Annotations
# \images

import os 
from tqdm import tqdm 

def check_data_match(dataset_root, move_, namestr="aqd",dir_={"images": "images", "labels": "annotations"}):
    img_dir = os.path.join(dataset_root, dir_["images"])
    lbl_dir = os.path.join(dataset_root, dir_["labels"])

    img_list = [f.split('.')[0] for f in tqdm(os.listdir(img_dir)) if f.endswith('.jpg')]
    lbl_list = [f.split('.')[0] for f in tqdm(os.listdir(lbl_dir)) if f.endswith('.xml')]

    unmatched_images = set(img_list) - set(lbl_list)
    unmatched_labels = set(lbl_list) - set(img_list)

    if unmatched_images or unmatched_labels:
        print("Unmatched images:", unmatched_images)
        print("Unmatched labels:", unmatched_labels)
    else:
        print("All images and labels match.")
        
    if move_ is not None:
        assert isinstance(move_, str), f"{move_} 得是一个输出文件夹"
        if not os.path.exists(move_):
            os.makedirs(move_, exist_ok=True)
        
        img_dir = os.path.join(move_, dir_["images"])
        lbl_dir = os.path.join(move_, dir_["labels"])
        
        img_list = [f.split('.')[0] for f in tqdm(os.listdir(img_dir)) if f.endswith('.jpg') and namestr in f]
        lbl_list = [f.split('.')[0] for f in tqdm(os.listdir(lbl_dir)) if f.endswith('.xml') and namestr in f]
        
data_root = r"D:\ddesktop\monitoring\datadata\safetyBelt\aichive\VOCDatasets"
check_data_match(dataset_root=data_root)

100%|██████████| 1694/1694 [00:00<?, ?it/s]

Unmatched images: set()
Unmatched labels: {'aqm_0046'}


In [ ]:
import shutil
def move_dataset_byStr(dataset_root, out_root, dir_={"images": "images", "labels": "annotations"}, theStr="aqd"):
    im_dir = os.path.join(dataset_root, dir_["images"])
    lb_dir = os.path.join(dataset_root, dir_["labels"])
    
    im_dir_save = os.path.join(out_root, dir_["images"])
    lb_dir_save = os.path.join(out_root, dir_["labels"]) 
    os.makedirs(im_dir_save, exist_ok=True)
    os.makedirs(lb_dir_save, exist_ok=True)
    
    im_namelist = [f.splt('.'[0]) for f in os.listdir(im_dir) if f.endswith('.jpg') and theStr in f]
    
    for f in tqdm(im_namelist, desc="Moving files"):
        img_name = f + '.jpg'
        xml_name = f + '.xml'
        if os.path.isfile(os.path.join(im_dir, img_name)) and os.path.isfile(os.path.join(lb_dir, xml_name)):
            
            # 复制图像文件和标注文件到指定的输出文件夹
            img_path = os.path.join(im_dir, img_name)
            img_path_save = os.path.join(im_dir_save, img_name)
            shutil.copyfile(img_path, img_path_save)
            
            xml_path = os.path.join(im_dir, img_name)
            xml_path_save = os.path.join(im_dir_save, xml_name)
            shutil.copyfile(xml_path, xml_path_save)
            
        else:
            print(f'no data {f}')
    

In [4]:
# move frames_2   D0x 录制视频文件 隔两秒取一帧  videofilenmae_i.jpg
#               -> D0x_dir
import shutil

def move_frames_to_subdir(frames_root, save_root):
    os.makedirs(save_root, exist_ok=True)
    
    # 获取所有jpg文件并开始处理
    for imfile in tqdm(os.listdir(frames_root)):
        if imfile.endswith('.jpg'):
            imgpath = os.path.join(frames_root, imfile)
            
            # 提取摄像头ID前缀
            camera_id = imfile.split('_')[0]
            save_dir = os.path.join(save_root, camera_id)
            
            # 创建子文件夹（如不存在）
            os.makedirs(save_dir, exist_ok=True)
            
            # 定义保存路径并复制文件
            impath_save = os.path.join(save_dir, imfile)
            shutil.copy2(imgpath, impath_save)  # 使用copy2保留文件的元数据
            
    print(f"All frames have been moved to subdirectories in {save_root}.")
    
frames_= r"D:\ddesktop\monitoring\datadata\frames_2"
save_ = r"D:\ddesktop\monitoring\datadata"
move_frames_to_subdir(frames_, save_)



100%|██████████| 8521/8521 [05:15<00:00, 27.04it/s]

All frames have been moved to subdirectories in D:\ddesktop\monitoring\datadata.


In [10]:
import random 
def select_and_move_images(source_dir, target_dir, num_images=800):
    os.makedirs(target_dir, exist_ok=True)

    # 获取所有 jpg 文件
    jpg_files = [f for f in os.listdir(source_dir) if f.endswith('.jpg')]
    
    # 检查是否有足够的文件
    if len(jpg_files) < num_images:
        print(f"Not enough images. Found {len(jpg_files)}, but need {num_images}.")
        return

    # 随机选择指定数量的文件
    selected_files = random.sample(jpg_files, num_images)

    # 移动选中的文件
    for filename in selected_files:
        # shutil.move(os.path.join(source_dir, filename), os.path.join(target_dir, filename))
        shutil.copyfile(os.path.join(source_dir, filename), os.path.join(target_dir, filename))

    print(f"{num_images} images have been moved to {target_dir}.")
    
data_dir = r"D:\ddesktop\monitoring\datadata\D02"  # D02 D03 D04 
save_dir = r"D:\ddesktop\monitoring\datadata\for_anno\D02"  # D02 D03 D04 

select_and_move_images(data_dir, save_dir,  500)
    

500 images have been moved to D:\ddesktop\monitoring\datadata\for_anno\D02.


In [14]:
# get frame  //3s 
import av
import cv2 
import numpy as np
import uuid 
import glob

def get_frames_with_pyav(video_root, output_dir, frame_interval=3, diff_threshold=0.5, std_threshold=15, range_threshold=20, filename_mode="file"):
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    video_path_list = glob.glob(os.path.join(video_root, '*.mp4'))
    # video_path_list = glob.glob(os.path.join(video_root, "**", '*.mp4'), recursive=True)
    # video_path_list = ["F:/drone/drone_data/videos/for_tracking/part0399c.mp4"]

    for idx, video_path in enumerate(tqdm(video_path_list, desc="Processing videos", unit="video")):
        try:
            tqdm.write(f"\n正在处理视频文件 ({idx + 1}/{len(video_path_list)}): {video_path}")
            container = av.open(video_path)

            stream = container.streams.video[0]
            fps = float(stream.average_rate)
            # print(f"{fps = }")
            if fps == 0:
                tqdm.write(f"无法获取视频 {video_path} 的帧率，已跳过。")
                continue

            frame_interval_frames = int(frame_interval * fps)
            frame_count = 0
            prev_frame = None

            # Extract video filename without extension
            video_filename = os.path.basename(video_path)
            video_name_no_ext = os.path.splitext(video_filename)[0]
            frame_save_count = 0

            for packet in container.demux(stream):
                for frame in packet.decode():
                    if frame_count % frame_interval_frames == 0:
                        frame_bgr = frame.to_ndarray(format='bgr24')

                        # 检查帧是否为乱码帧
                        frame_gray_full = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)
                        std_dev = np.std(frame_gray_full)
                        min_val, max_val = frame_gray_full.min(), frame_gray_full.max()
                        pixel_range = max_val - min_val

                        if (std_dev < std_threshold and pixel_range < range_threshold):
                            continue  # 跳过乱码帧

                        if prev_frame is not None:
                            prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
                            frame_gray = frame_gray_full
                            difference = np.mean(np.abs(prev_frame_gray.astype("float") - frame_gray.astype("float")))

                            if difference > diff_threshold:
                                if filename_mode == "uuid":
                                    # 保存帧
                                    _name = f"{uuid.uuid4()}.jpg"
                                else:
                                    # filename_mode = "file"
                                    _name = f"{video_name_no_ext}_{frame_save_count}.jpg"

                                output_path = os.path.join(output_dir, _name)
                                cv2.imwrite(output_path, frame_bgr)
                                frame_save_count += 1
                                prev_frame = frame_bgr
                        else:
                            # 保存第一帧
                            if filename_mode == "uuid":
                                _name = f"{uuid.uuid4()}.jpg"
                            else:
                                # filename_mode = "file"
                                _name = f"{video_name_no_ext}_{frame_save_count}.jpg"

                            output_path = os.path.join(output_dir, _name)
                            cv2.imwrite(output_path, frame_bgr)
                            frame_save_count += 1
                            prev_frame = frame_bgr

                    frame_count += 1

            container.close()

        except Exception as e:
            tqdm.write(f"无法处理视频 {video_path}:{e}") 
            continue

vroot = r"E:\nobelt"
out_ = r"D:\ddesktop\monitoring\datadata\nobelt"
    
get_frames_with_pyav(vroot, out_)


                                          
  0%|          | 0/1693 [4:44:14<?, ?it/s]                 


正在处理视频文件 (1/5): E:\nobelt\D02_20241104143825.mp4
fps = 25.0


                                          
  0%|          | 0/1693 [4:46:56<?, ?it/s]                          


正在处理视频文件 (2/5): E:\nobelt\D03_20241104143825.mp4
fps = 25.0


                                          
  0%|          | 0/1693 [4:47:08<?, ?it/s]                         

...

                                          
  0%|          | 0/1693 [4:51:36<?, ?it/s]                          


正在处理视频文件 (4/5): E:\nobelt\D04_20241104143825.mp4
fps = 25.0


                                          
  0%|          | 0/1693 [4:55:00<?, ?it/s]                          


正在处理视频文件 (5/5): E:\nobelt\D04_20241104145206.mp4
fps = 25.0


Processing videos: 100%|██████████| 5/5 [11:22<00:00, 136.50s/video]


In [ ]:
# check od dataset 
def del_notmatch_data(od_dataset_root, data_struct={"image": "images", "label": "labels"}, delete_=False):
    img_dir = os.path.join(od_dataset_root, data_struct["image"])
    lbl_dir = os.path.join(od_dataset_root, data_struct["label"])
    img_name_list = [f.split('.')[0] for f in os.listdir(img_dir) if f.endswith('.jpg')]
    lbl_name_list = [f.split('.')[0] for f in os.listdir(lbl_dir) if f.endswith('.xml')]
    
    img_no_label_list = [f for f in img_name_list if f not in lbl_name_list]
    print(f"{img_no_label_list=}")
    label_no_img_list = [f for f in lbl_name_list if f not in img_name_list]
    print(f"{label_no_img_list=}")
    
    if delete_:
        # delet
        for t in img_no_label_list:
            im_path = os.path.join(img_dir, t+'.jpg')
            if os.path.isfile(im_path):
                # print('can delete jpg file')
                print(f"delete {os.path.basename(im_path)}")
                os.remove(im_path)
        
        for t in label_no_img_list:
            xml_path = os.path.join(lbl_dir, t+'.xml')
            if os.path.isfile(xml_path):
                # print('can delete xml file')
                print(f"delete {os.path.basename(xml_path)}")
                os.remove(xml_path)
    
dataset_ = r"D:\ddesktop\monitoring\datadata\belthelmet\1104"
del_notmatch_data(dataset_)

In [ ]:
# d_root = r"D:\ddesktop\monitoring\codespace\yolov8_BeltHelmet\VOCdevkit\VOC2007"
# del_notmatch_data(d_root, data_struct={"image": "JPEGImages", "label": "Annotations"}, delete_=False)
# # ok match

In [ ]:
# del empty data
import xml.etree.ElementTree as ET 
def del_empty_xml_file_and_image(image_dir, xml_dir):
    """
    删除 xml_dir 中没有任何 bbox 的 XML 文件，以及对应的图像文件。

    参数：
        image_dir (str): 存放图像文件（JPG）的目录路径。
        xml_dir (str): 存放 XML 标注文件的目录路径。
    """
    deleted_xml_files = []
    deleted_image_files = []
    total_xml_files = 0

    for xml_file in os.listdir(xml_dir):
        if xml_file.endswith('.xml'):
            total_xml_files += 1
            xml_path = os.path.join(xml_dir, xml_file)
            try:
                tree = ET.parse(xml_path)
                root = tree.getroot()
                # 查找所有的 object 元素
                objects = root.findall('object')
                if not objects:
                    # 如果没有找到 object 元素，删除该 XML 文件和对应的图像文件
                    os.remove(xml_path)
                    deleted_xml_files.append(xml_file)
                    print(f"已删除空的 XML 文件：{xml_file}")

                    # 对应的图像文件名
                    base_name = os.path.splitext(xml_file)[0]
                    # 尝试匹配不同扩展名的图像文件
                    image_extensions = ['.jpg', '.jpeg', '.png']
                    image_deleted = False
                    for ext in image_extensions:
                        image_file = base_name + ext
                        image_path = os.path.join(image_dir, image_file)
                        if os.path.exists(image_path):
                            os.remove(image_path)
                            deleted_image_files.append(image_file)
                            print(f"已删除对应的图像文件：{image_file}")
                            image_deleted = True
                            break
                    if not image_deleted:
                        print(f"未找到对应的图像文件：{base_name}.[jpg|jpeg|png]")
            except ET.ParseError:
                # 如果 XML 文件解析错误，也删除文件和对应的图像文件
                os.remove(xml_path)
                deleted_xml_files.append(xml_file)
                print(f"已删除无法解析的 XML 文件：{xml_file}")

                # 同样删除对应的图像文件
                base_name = os.path.splitext(xml_file)[0]
                image_extensions = ['.jpg', '.jpeg', '.png']
                image_deleted = False
                for ext in image_extensions:
                    image_file = base_name + ext
                    image_path = os.path.join(image_dir, image_file)
                    if os.path.exists(image_path):
                        os.remove(image_path)
                        deleted_image_files.append(image_file)
                        print(f"已删除对应的图像文件：{image_file}")
                        image_deleted = True
                        break
                if not image_deleted:
                    print(f"未找到对应的图像文件：{base_name}.[jpg|jpeg|png]")

    print(f"总共扫描了 {total_xml_files} 个 XML 文件，删除了 {len(deleted_xml_files)} 个空的 XML 文件。")
    print(f"删除了 {len(deleted_image_files)} 个对应的图像文件。")

img_d = r"D:\ddesktop\monitoring\datadata\belthelmet\1104\images"
xml_d = r"D:\ddesktop\monitoring\datadata\belthelmet\1104\labels"
del_empty_xml_file_and_image(img_d, xml_d)


In [ ]:
# od dataset crop

# -> list: x1,y1,x2,y2
def read_bbox(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    bboxes = []
    for obj in root.findall('object'):
        
        # # 获取目标类别名称，如果需要，可以根据类别名称进行过滤
        # name = obj.find('name').text
        # if name == "drone":
        
        bbox = obj.find('bndbox')
        xmin = int(float(bbox.find('xmin').text))
        ymin = int(float(bbox.find('ymin').text))
        xmax = int(float(bbox.find('xmax').text))
        ymax = int(float(bbox.find('ymax').text))
        bboxes.append((xmin, ymin, xmax, ymax))
    return bboxes
        
def get_16w9h_crop_box_random(xmin, xmax, ymin, ymax, img_w, img_h, min_wsize, wdh=16/9, iou_th=0.6):
    """
    生成一个随机裁剪框，宽高比为16:9，包含目标对象，
    且裁剪区域与目标对象的交叉面积至少为 iou_th * bbox_area。

    参数：
        xmin, xmax, ymin, ymax: 目标对象的边界框坐标。
        img_w, img_h: 图像的宽度和高度。
        min_wsize: 裁剪框的最小宽度（默认值为1080）。
        wdh: 宽高比（默认值为16/9）。
        iou_th: 裁剪区域与目标对象的最小交叉面积比例。

    返回：
        x1, x2, y1, y2: 裁剪框的左上角和右下角坐标。
    """

    bbox_w = xmax - xmin
    bbox_h = ymax - ymin
    bbox_area = bbox_w * bbox_h

    # 计算裁剪框的最小宽度和高度，确保包含目标对象并满足最小尺寸
    crop_w_min = max(bbox_w, min_wsize)
    crop_h_min = crop_w_min / wdh

    # 确保裁剪框尺寸不超过图像尺寸
    crop_w_max = min(img_w, img_h * wdh)
    crop_h_max = crop_w_max / wdh

    max_attempts = 100  # 最大尝试次数
    for _ in range(max_attempts):
        # 在最小和最大尺寸之间随机选择裁剪宽度
        crop_w = random.uniform(crop_w_min, crop_w_max)
        crop_h = crop_w / wdh

        # 计算裁剪框的左上角坐标范围，确保包含目标对象且在图像范围内
        x1_min = max(0, xmin - (crop_w - bbox_w))
        x1_max = min(xmin, img_w - crop_w)
        y1_min = max(0, ymin - (crop_h - bbox_h))
        y1_max = min(ymin, img_h - crop_h)

        if x1_min > x1_max or y1_min > y1_max:
            continue  # 无法找到合适的裁剪框，继续下一次尝试

        # 在计算得到的范围内随机选择左上角坐标
        x1 = random.uniform(x1_min, x1_max)
        y1 = random.uniform(y1_min, y1_max)
        x2 = x1 + crop_w
        y2 = y1 + crop_h

        # 计算裁剪框与目标对象的交叉面积
        inter_x1 = max(x1, xmin)
        inter_y1 = max(y1, ymin)
        inter_x2 = min(x2, xmax)
        inter_y2 = min(y2, ymax)

        if inter_x1 < inter_x2 and inter_y1 < inter_y2:
            inter_area = (inter_x2 - inter_x1) * (inter_y2 - inter_y1)
            if inter_area >= iou_th * bbox_area:
                # 返回整数坐标
                return int(x1), int(x2), int(y1), int(y2)

    # 如果未找到合适的裁剪框，返回原始边界框
    return int(xmin), int(xmax), int(ymin), int(ymax)

def cal_iou(x1, y1, x2, y2, ax1, ay1, ax2, ay2):
    """
    计算裁剪框与目标边界框的交叉面积比例。

    参数：
        x1, y1, x2, y2: 裁剪框的坐标。
        ax1, ay1, ax2, ay2: 目标边界框的坐标。

    返回：
        iou: 交叉面积与目标边界框面积的比例。
    """
    bbox_area = (ax2 - ax1) * (ay2 - ay1)
    inter_x1 = max(x1, ax1)
    inter_y1 = max(y1, ay1)
    inter_x2 = min(x2, ax2)
    inter_y2 = min(y2, ay2)

    if inter_x1 < inter_x2 and inter_y1 < inter_y2:
        inter_area = (inter_x2 - inter_x1) * (inter_y2 - inter_y1)
        iou_ = inter_area / bbox_area
        return iou_
    else:
        return 0.0

def save_cropped_xml(original_xml_path, save_path, crop_box, new_image_size, new_image_name):
    tree = ET.parse(original_xml_path)
    root = tree.getroot()
    root.find('filename').text = new_image_name
    size = root.find('size')
    size.find('width').text = str(new_image_size[0])
    size.find('height').text = str(new_image_size[1])
    x1_crop, x2_crop, y1_crop, y2_crop = crop_box
    objects = root.findall('object')
    for obj in objects:
        bbox = obj.find('bndbox')
        xmin = int(float(bbox.find('xmin').text))
        ymin = int(float(bbox.find('ymin').text))
        xmax = int(float(bbox.find('xmax').text))
        ymax = int(float(bbox.find('ymax').text))
        xmin_new = xmin - x1_crop
        ymin_new = ymin - y1_crop
        xmax_new = xmax - x1_crop
        ymax_new = ymax - y1_crop
        if xmin_new < 0 or ymin_new < 0 or xmax_new > new_image_size[0] or ymax_new > new_image_size[1]:
            root.remove(obj)
            continue
        bbox.find('xmin').text = str(max(0, xmin_new))
        bbox.find('ymin').text = str(max(0, ymin_new))
        bbox.find('xmax').text = str(min(new_image_size[0], xmax_new))
        bbox.find('ymax').text = str(min(new_image_size[1], ymax_new))
    new_xml_name = os.path.join(save_path, f"{os.path.splitext(new_image_name)[0]}.xml")
    tree.write(new_xml_name)

def crop_images_with_xml(image_dir, xml_dir, save_dir, add_='a', min_wsize_= 1080):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    cropped_xml_save_dir = os.path.join(save_dir, "Annotations")
    if not os.path.exists(cropped_xml_save_dir):
        os.makedirs(cropped_xml_save_dir)
    cropped_image_save_dir = os.path.join(save_dir, "JPEGImages")
    if not os.path.exists(cropped_image_save_dir):
        os.makedirs(cropped_image_save_dir)
    for xml_file in tqdm(os.listdir(xml_dir)):
        if xml_file.endswith('.xml'):
            xml_path = os.path.join(xml_dir, xml_file)
            image_file = xml_file.replace('.xml', '.jpg')
            image_path = os.path.join(image_dir, image_file)
            if not os.path.exists(image_path):
                print(f"Image file {image_path} not found.")
                continue
            
            image = Image.open(image_path)
            w, h = image.size
            
            if max(w, h) < min_wsize_:
                continue
            
            bounding_boxes = read_bbox(xml_path)
            if not bounding_boxes:
                print(f"No bounding boxes found in {xml_path}.")
                continue
            for idx, (xmin, ymin, xmax, ymax) in enumerate(bounding_boxes):
                x1, x2, y1, y2 = get_16w9h_crop_box_random(xmin, xmax, ymin, ymax, w, h, min_wsize_)
                cropped_image = image.crop((x1, y1, x2, y2))
                cropped_image_name = f"{os.path.splitext(image_file)[0]}_{add_}_{idx}.jpg"
                cropped_image.save(os.path.join(cropped_image_save_dir, cropped_image_name))
                save_cropped_xml(xml_path, cropped_xml_save_dir, (x1, x1, y1, y1), (x2 - x1, y2 - y1), cropped_image_name)
                # print(f"Cropped and saved {cropped_image_name} and corresponding XML.")

def crop_xml_od_dataset(image_dir, xml_dir, save_dir):
    crop_images_with_xml(image_dir, xml_dir, save_dir)

image_dir = r"D:\ddesktop\monitoring\datadata\belthelmet\1104\images"
xml_dir = r"D:\ddesktop\monitoring\datadata\belthelmet\1104\labels"
save_dir = r"D:\ddesktop\monitoring\datadata\belthelmet\1104_expand"
crop_images_with_xml(image_dir, xml_dir, save_dir)


In [ ]:
# del empty data
def del_empty_xml_file_and_image(image_dir, xml_dir):
    """
    删除 xml_dir 中没有任何 bbox 的 XML 文件，以及对应的图像文件。

    参数：
        image_dir (str): 存放图像文件（JPG）的目录路径。
        xml_dir (str): 存放 XML 标注文件的目录路径。
    """
    deleted_xml_files = []
    deleted_image_files = []
    total_xml_files = 0

    for xml_file in os.listdir(xml_dir):
        if xml_file.endswith('.xml'):
            total_xml_files += 1
            xml_path = os.path.join(xml_dir, xml_file)
            try:
                tree = ET.parse(xml_path)
                root = tree.getroot()
                # 查找所有的 object 元素
                objects = root.findall('object')
                if not objects:
                    # 如果没有找到 object 元素，删除该 XML 文件和对应的图像文件
                    os.remove(xml_path)
                    deleted_xml_files.append(xml_file)
                    print(f"已删除空的 XML 文件：{xml_file}")

                    # 对应的图像文件名
                    base_name = os.path.splitext(xml_file)[0]
                    # 尝试匹配不同扩展名的图像文件
                    image_extensions = ['.jpg', '.jpeg', '.png']
                    image_deleted = False
                    for ext in image_extensions:
                        image_file = base_name + ext
                        image_path = os.path.join(image_dir, image_file)
                        if os.path.exists(image_path):
                            os.remove(image_path)
                            deleted_image_files.append(image_file)
                            print(f"已删除对应的图像文件：{image_file}")
                            image_deleted = True
                            break
                    if not image_deleted:
                        print(f"未找到对应的图像文件：{base_name}.[jpg|jpeg|png]")
            except ET.ParseError:
                # 如果 XML 文件解析错误，也删除文件和对应的图像文件
                os.remove(xml_path)
                deleted_xml_files.append(xml_file)
                print(f"已删除无法解析的 XML 文件：{xml_file}")

                # 同样删除对应的图像文件
                base_name = os.path.splitext(xml_file)[0]
                image_extensions = ['.jpg', '.jpeg', '.png']
                image_deleted = False
                for ext in image_extensions:
                    image_file = base_name + ext
                    image_path = os.path.join(image_dir, image_file)
                    if os.path.exists(image_path):
                        os.remove(image_path)
                        deleted_image_files.append(image_file)
                        print(f"已删除对应的图像文件：{image_file}")
                        image_deleted = True
                        break
                if not image_deleted:
                    print(f"未找到对应的图像文件：{base_name}.[jpg|jpeg|png]")

    print(f"总共扫描了 {total_xml_files} 个 XML 文件，删除了 {len(deleted_xml_files)} 个空的 XML 文件。")
    print(f"删除了 {len(deleted_image_files)} 个对应的图像文件。")

img_d = r"D:\ddesktop\monitoring\datadata\belthelmet\1104_expand\JPEGImages"
xml_d = r"D:\ddesktop\monitoring\datadata\belthelmet\1104_expand\Annotations"
del_empty_xml_file_and_image(img_d, xml_d)


In [ ]:
import os
import shutil
import random

def sample_files(data_dir, target_dir, samples_=40):
    """
    从 data_dir 中随机抽取 samples_ 个文件，复制到 target_dir。

    参数：
        data_dir (str): 源文件夹路径。
        target_dir (str): 目标文件夹路径。
        samples_ (int): 要抽取的文件数量，默认为60。
    """
    # 如果目标目录不存在，则创建
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    # 获取 data_dir 中的所有文件列表
    files = [f for f in os.listdir(data_dir) if os.path.isfile(os.path.join(data_dir, f))]
    
    # 检查是否有足够的文件进行抽样
    if len(files) < samples_:
        print(f"文件数量不足，共有 {len(files)} 个文件，无法抽取 {samples_} 个样本。")
        return

    # 随机抽取 samples_ 个文件
    sampled_files = random.sample(files, samples_)

    # 复制文件到 target_dir
    for filename in sampled_files:
        src_path = os.path.join(data_dir, filename)
        dst_path = os.path.join(target_dir, filename)
        shutil.copy2(src_path, dst_path)  # 使用 copy2 保留文件元数据

    print(f"已成功复制 {samples_} 个文件到 {target_dir}。")


dir_ = r"D:\ddesktop\monitoring\datadata\for_anno\D04"
save_dir = r"D:\ddesktop\monitoring\datadata\belthelmet\1106_1309\images"

sample_files(dir_, save_dir)

In [ ]:
def move_jpg_by_xml(xml_dir, jpg_dir, jpg_save_dir):
    name_list = [f.split('.xml')[0] for f in os.listdir(xml_dir) if f.endswith('.xml')]
    for fs in name_list:
        im_name = fs + '.jpg'
        im_path = os.path.join(jpg_dir, im_name)
        if os.path.isfile(im_path):
            im_path_save = os.path.join(jpg_save_dir, im_name)
            shutil.copy2(im_path, im_path_save) 

x_dir = r"D:\ddesktop\monitoring\datadata\belthelmet\1106_1309\labels"
im_dir = r"D:\ddesktop\monitoring\datadata\for_anno\nobelt"
im_save_dir = r"D:\ddesktop\monitoring\datadata\belthelmet\1106_1309\images"
move_jpg_by_xml(x_dir, im_dir, im_save_dir)
        

In [2]:
# xml info
import os
import xml.etree.ElementTree as ET
from collections import defaultdict

def analysis_xml(xml_dir, output_file=None):
    # 字典存储各个类别的计数
    class_count = defaultdict(int)
    # 字典存储各个分辨率的图像数量
    resolution_count = defaultdict(int)
    # 字典存储各种尺寸的标注框数量
    bbox_size_count = defaultdict(int)

    # 遍历指定目录下的所有 XML 文件
    for xml_file in os.listdir(xml_dir):
        if xml_file.endswith('.xml'):
            # 构建完整的文件路径
            path = os.path.join(xml_dir, xml_file)
            # 解析 XML 文件
            tree = ET.parse(path)
            root = tree.getroot()
            
            # 提取图像的分辨率
            width = root.find('size/width').text
            height = root.find('size/height').text
            resolution = f'{width}x{height}'
            resolution_count[resolution] += 1

            # 遍历所有的 object 标签
            for obj in root.findall('object'):
                class_name = obj.find('name').text
                class_count[class_name] += 1
                
                # 获取 bounding box 尺寸
                bndbox = obj.find('bndbox')
                xmin = int(bndbox.find('xmin').text)
                ymin = int(bndbox.find('ymin').text)
                xmax = int(bndbox.find('xmax').text)
                ymax = int(bndbox.find('ymax').text)
                bbox_width = xmax - xmin
                bbox_height = ymax - ymin
                bbox_size = f'{bbox_width}x{bbox_height}'
                bbox_size_count[bbox_size] += 1

    # 打印结果
    print("类别及其数量:")
    for cls, count in class_count.items():
        print(f'{cls}: {count}')
    
    print("\n分辨率及其图像数量:")
    for res, count in resolution_count.items():
        # print(f'{res}: {count}')
        continue
    
    print("\n标注框尺寸及其数量:")
    for size, count in bbox_size_count.items():
        # print(f'{size}: {count}')
        continue

    if output_file is not None:
        # 打开文件并写入数据
        with open(output_file, 'w', encoding='utf-8') as file:
            file.write("类别及其数量:\n")
            for cls, count in class_count.items():
                file.write(f'{cls}: {count}\n')
            
            file.write("\n分辨率及其图像数量:\n")
            for res, count in resolution_count.items():
                file.write(f'{res}: {count}\n')

            file.write("\n标注框尺寸及其数量:\n")
            for size, count in bbox_size_count.items():
                file.write(f'{size}: {count}\n')

xml_dir = r"D:\ddesktop\monitoring\datadata\belthelmet\dataset\labels"

txt_path = os.path.join(os.path.dirname(xml_dir), "datasetinfo.txt")
# 调用函数
analysis_xml(xml_dir, txt_path)


类别及其数量:
helmet: 6753
safetybelt: 2629
head: 349

分辨率及其图像数量:

标注框尺寸及其数量:


In [ ]:
def sample_od_dataset(dataset_root, target_root, _struct={"image": "images", "label": "labels"}, samples_=100):
    image_dir = os.path.join(dataset_root, _struct["image"])
    label_dir = os.path.join(dataset_root, _struct["label"])

    # 获取图像和标注的文件名（不含扩展名）
    im_namelist = [os.path.splitext(f)[0] for f in os.listdir(image_dir) if f.endswith('.jpg')]
    la_namelist = [os.path.splitext(f)[0] for f in os.listdir(label_dir) if f.endswith('.xml')]

    # 确保目标目录存在
    image_dir_save = os.path.join(target_root, _struct["image"])
    label_dir_save = os.path.join(target_root, _struct["label"])
    os.makedirs(image_dir_save, exist_ok=True)
    os.makedirs(label_dir_save, exist_ok=True)

    # 获取图像和标注文件的交集，确保图像和标注匹配
    namelist = [f for f in im_namelist if f in la_namelist]

    # 检查可用样本数是否足够
    if len(namelist) < samples_:
        print(f"可用的样本数只有 {len(namelist)}，少于请求的 {samples_} 个样本。将使用全部可用样本。")
        samples_ = len(namelist)

    # 随机打乱并抽取所需数量的样本
    random.shuffle(namelist)
    target_namelist = namelist[:samples_]

    for t in target_namelist:
        im_path = os.path.join(image_dir, t + '.jpg')
        im_path_save = os.path.join(image_dir_save, t + '.jpg')
        shutil.copy2(im_path, im_path_save)

        xml_path = os.path.join(label_dir, t + '.xml')
        xml_path_save = os.path.join(label_dir_save, t + '.xml')
        shutil.copy2(xml_path, xml_path_save)

    print(f"已成功复制 {samples_} 个样本到 {target_root}。")
    